In [1]:
import tensorcircuit as tc
from tensorcircuit import Circuit

import numpy as np

Please first ``pip install -U cirq`` to enable related functionality in translation module


In [2]:
from tensorcircuit.cloud import apis
apis.set_token("ENTER YOUR TOKEN HERE")

# The following is a list of avaliable QPU devices.
apis.list_devices(provider="tencent")

[tencent::tianji_s2v6,
 tencent::simulator:tc,
 tencent::tianji_m2v14s2,
 tencent::tianji_m1v15s1,
 tencent::tianji_m1v15s2,
 tencent::tianji_m2,
 tencent::tianji_m1v15s3,
 tencent::tianji_m1v15s4,
 tencent::tianji_s2v7,
 tencent::tianji_m2v16s1,
 tencent::tianji_m2v14s4,
 tencent::tianji_m1,
 tencent::tianji_s2,
 tencent::tianxuan_s1,
 tencent::tianji_m2v15s3,
 tencent::tianxuan_s2v20]

Determine QPU devices.

In [3]:
QPU_DEVICE = "tianji_s2v6"

In [4]:
from angle.wx_angles import quantum_signal_processing
from numpy.polynomial.polynomial import Polynomial
from numpy.polynomial.chebyshev import cheb2poly

## Algorithm

**Input**: $k$ copies of an evolution dynamic $U_{H}(\theta) = e^{-iH \theta}$, where $k$ is odd, $t \in \mathbb{R}$ is unknown and $H$ is an unknown Hamiltonian from a known set $\{X, Z\}$.

**Output**: 1 bit as a guess of $H$, where '0' stands for $Z$ and '1' stands for $X$.

1. Determine the vector of angles $\vec{\phi} = \phi_0, \ldots, \phi_k$. 
2. Construct the single-qubit circuit ${\rm QSP}_{H,k}(\theta)$.
3. Input the zero state $\ket{0}$, run the circuit. Then perform a computational basis measurement at the end, and return the measurement outcome.

## Input setting

In [5]:
num_shot = 64
total_shots = 1024

In [6]:
list_theta = np.linspace(0, 2 * np.pi, num_shot, endpoint=False)

Prepare angle

In [7]:
def phi_set(k: int) -> np.ndarray:
    r"""Compute the angle for the QSP circuit.
    
    Args:
        k: depth
        
    Returns:
        phi angles for Wx-based QSP
    
    """
    chebyshev_coef = np.zeros([k + 1])
    for i in range(k + 1):
        if i % 2 == 1:
            chebyshev_coef[i] = -1
    chebyshev_coef = chebyshev_coef * 2 / (k + 1)
    
    P = Polynomial(cheb2poly(chebyshev_coef))
    return quantum_signal_processing(P)

Prepare Algorithm

In [8]:
def _qsp(label: str, list_phi: np.ndarray, theta: np.ndarray) -> Circuit:
    cir = Circuit(1)
    cir.rz(0, theta=-2 * list_phi[0])
    
    for phi in list_phi[1:]:
        if label == '0':
            cir.rz(0, theta=theta)
        else:
            cir.rx(0, theta=theta)
        cir.rz(0, theta=-2 * phi)
    return cir

def algorithm1(label: str, k: int) -> np.ndarray:
    r"""Implementation of the algorithm 1 that runs 64 times, theta is uniformly sampled from [0, 2 * pi].
    
    Args:
        label: label of the input unknown evolution operator; 0, 1 stand for Z and X, respectively.
        k: number of available copies
    
    Returns:
        the experimental probability of correctly guessing the label
    
    """
    if num_shot > 64:
        raise NotImplementedError(
            "The number of shots is too large for Tencent Cloud platform.")

    list_phi = phi_set(k)
    list_cir = [_qsp(label, list_phi, theta) for theta in list_theta]

    ts = apis.submit_task(
        circuit=list_cir,
        # provider="local",
        # device="testing",
        device=QPU_DEVICE,
        shots=total_shots,
    )
    correct_shots = np.array([(t.results()[label] if label in t.results() else 0) for t in ts])
    return correct_shots / total_shots

## Begin experiment

In [9]:
def experiment(k: int) -> int:
    r"""Perform the experiment for a given k
    
    Args:
        k: number of available copies
    
    Returns:
        the experimental probability of success
    
    """
    assert k % 2 == 1, "k must be odd"

    list_label = ['0', '1']
    return np.stack([algorithm1(label, k) for label in list_label])

In [10]:
list_k = np.array([3, 5, 7, 9, 11, 13, 15])
ideal_success = ((2 * list_k + 1) / (2 * list_k + 2))

list_experiment_success = np.stack([experiment(k) for k in list_k])

c:\Users\Cloud\Hamiltonian-sensing\angle\wx_angles.py:190: UserWarning: Phi verification failed: y - P(x) = (-3.900344329732519e-07-1.350558782482747e-06j), error = 1.4057510035017033e-06
  warnings.warn("Phi verification failed: " +
c:\Users\Cloud\Hamiltonian-sensing\angle\wx_angles.py:190: UserWarning: Phi verification failed: y - P(x) = (-6.974535588233355e-07-7.585115738232423e-07j), error = 1.0304277142716747e-06
  warnings.warn("Phi verification failed: " +


In [11]:
list_theta

array([0.        , 0.09817477, 0.19634954, 0.29452431, 0.39269908,
       0.49087385, 0.58904862, 0.68722339, 0.78539816, 0.88357293,
       0.9817477 , 1.07992247, 1.17809725, 1.27627202, 1.37444679,
       1.47262156, 1.57079633, 1.6689711 , 1.76714587, 1.86532064,
       1.96349541, 2.06167018, 2.15984495, 2.25801972, 2.35619449,
       2.45436926, 2.55254403, 2.6507188 , 2.74889357, 2.84706834,
       2.94524311, 3.04341788, 3.14159265, 3.23976742, 3.33794219,
       3.43611696, 3.53429174, 3.63246651, 3.73064128, 3.82881605,
       3.92699082, 4.02516559, 4.12334036, 4.22151513, 4.3196899 ,
       4.41786467, 4.51603944, 4.61421421, 4.71238898, 4.81056375,
       4.90873852, 5.00691329, 5.10508806, 5.20326283, 5.3014376 ,
       5.39961237, 5.49778714, 5.59596191, 5.69413668, 5.79231146,
       5.89048623, 5.988661  , 6.08683577, 6.18501054])

In [12]:
ideal_success

array([0.875     , 0.91666667, 0.9375    , 0.95      , 0.95833333,
       0.96428571, 0.96875   ])

In [13]:
list_experiment_success.shape

(7, 2, 64)

In [14]:
np.save("tc_k.npy", list_k)
np.save("tc_theta.npy", list_theta)
np.save("tc_xz.npy", list_experiment_success)